In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# 양자화 매개변수 정의
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # 4비트 양자화
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [3]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Evo"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
!nvidia-smi

Tue Oct 15 00:26:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0              30W /  70W |   5559MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
device = "cuda"

messages = [
    {
        "role": "system",
        "content": "당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.",
    },
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해줘"},
]

# 채팅 템플릿 생성
encoded = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)


model_inputs = encoded.to(device)

# 답변 종료 토큰 설정
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# 모델 입력
generated_ids = model.generate(
    model_inputs,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    repetition_penalty=1.05,
)

decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

금리 안정 정책으로 은행의 기준 금리에 대해 설명하겠습니다.

은행의 기준 금리는 시중 은행들이 저축성 예금과 기업들로부터의 대출에 내면에 적용하는 금리입니다. 이는 우리가 흔히 말하는 연 이자율입니다. 기준 금리는 중앙은행이 통제함으로써 부동산 기대감이나 물가 등 경제 상황에 따라 변동합니다.

그런데 이 기준 금리가 매우 중요합니다. 왜냐하면 기준 금리를 바꿔는 것만으로도 경제의 흐름을 바꿀 수 있다는 것입니다.

1. 소비자 지출: 기준 금리가 오르더라도 은행에서 대출한 돈에 적용되는 이자율도 오른다고 생각하면 됩니다. 기준 금리가 높아지면 금리를 올리는 효과를 줍니다. 이렇게 되면 소비자가 어떻게든지 예전에 소비했던 것처럼 다시 쇼핑하거나 투자 등에 몰릴 수도 있습니다.
2. 대출: 낮은 기준 금리에 의해서 다국적 대기업이나 개인들은 신용도가 좋아져서 대출을 받아내다. 이로 인해 기업들은 보다 많은 자금을 대출할 수 있고, 소비자들이도 대출의 이자를 내지 않고도 소비할 수도 있습니다.

그러나 기준 금릿의 오름은 금융이 마비되는 요인일 것이고 경제 성장과는 직접 반비례하는 상황이 많습니다.

현재 기준 금리의 조치에는 두가지 방도가 있습니다. 먼저 소폭 올리기인데 이것은 시중에 유입되는 자금이 늘어날 것이라고 예상됩니다. 또 다른 한 가지 조치는 그것을 줄이는데 이것은 소비자들의 자금부담이 줄어들 수 있습니다.

하지만 이러한 기준 금리의 조치는 전문가인 정기세미나가 필요합니다. 그렇기로 하여금 조치에 대한 고민을 가집니다.<|end_of_text|

In [10]:
# 검색 결과에 참고할 문서를 넣어서 문서 내에서 데이터를 찾는지 실험
user_prompt = "해군이 쏘카와 도입하는 서비스는?"

messages = [
    {
        "role": "system",
        "content": "당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 인공지능 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.",
    },
    {"role": "user", "content": user_prompt},
]

# 채팅 템플릿 생성
encodeds = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)

# 모델에 입력할 데이터를 gpu로 옮긴다.
model_inputs = encodeds.to(device)

# 답변 종료 토큰 설정
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# 모델 입력
generated_ids = model.generate(
    model_inputs,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    repetition_penalty=1.05,
)

decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 인공지능 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

해군이 쏘카와 도입하는 서비스는?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

'해양검색'의 주제에 맞는 질문을 보내신 것 같군요. 해군이 쏘카(SOCAR)와 도입하는 서비스가 궁금하시군요. 

SOCCAR는 소셜 페이스북에서 소셜 네트워킹을 위한 자사 서비스라고 합니다.  해군이 SOCCAR를 도입한다고 하면 매우 이례적인 일이겠지만, 이 사실과 관련된 정보는 없습니다.

(해군이나 쏘카/도입이 확인된다면 이에 기초하여 답변합니다.)<|end_of_text|>


In [11]:
# 참고문서가 주어짐

search_result = """쏘카(대표 이재웅)가 해군(해군참모총장 심승섭 대장)과 공유경제 활성화 및 업무 효율 향상을 위한 업무 협약을 체결했다. 해군은 국군 중 최초로 법인용 차량 공유 서비스 '쏘카 비즈니스'를 도입한다. 쏘카와 해군은 지난 5일 서울 해군 재경근무지원대대 회의실에서 김남희 쏘카 신규사업본부장과 조정권 해군본부 군수차장과 김은석 해군본부 수송과장 등 주요 관계자가 참석한 가운데 상호협력과 발전을 위한 업무 협약식을 체결했다. 양 기관은 △해군본부 임직원의 업무 이동 효율성 향상 △공유 차량을 활용한 해군 본부 및 부대 주차난 해소 △공유 차량 이용 활성화 및 확대를 위해 적극 협력키로 했다. 쏘카는 해군 장병과 군무원을 대상으로 법인용 차량 공유 서비스 '쏘카 비즈니스'를 제공한다. 해군 장병들과 군무원은 업무 이동 시 전국 쏘카존에 있는 1만 2천여 대의 차량을 이용할 수 있다. 특히, 출장 시에는 전국 74개 시군의 KTX, 기차역, 버스터미널, 공항 등 대중교통과 교통 편의시설 거점이 연결된 260여 개 쏘카존을 통해 효율적인 이동이 가능해진다. 쏘카와 해군은 우선 올해까지 해군본부를 대상으로 '쏘카 비즈니스' 시범 적용을 거친 후 내년부터는 해군 전 부대로 확대할 계획이다. 그전까지 일반 사병들에게는 별도로 월별 할인 혜택과 특전을 제공, 휴가와 외출 시에도 합리적인 가격으로 쏘카를 이용할 수 있도록 지원한다. 조정권 해군본부 군수차장은 "해군은 장병들의 업무 이동 편의성을 향상시키는 동시에 사기진작과 복리 증진을 위해 차량 공유 서비스 기업과 업무협약을 체결했다"며 "전문기관, 업체와의 협력을 통해 새로운 기술을 해군 수송업무에 도입해 해군이 그려나가는 '스마트 해군' 건설에 한 걸음 더 다가갈 수 있도록 노력하겠다"고 말했다. 김남희 쏘카 신규사업본부장은 "일반 기업체 외에도 군이나 지자체, 공공기관 등에서도 법인용 차량 공유 서비스에 대한 수요가 꾸준히 늘어나고 있다"며 "업무 이용 패턴과 특성에 맞는 서비스 인프라와 라인업을 지속해서 확대해 나갈 것"이라고 말했다."""
user_prompt = f"""검색 결과:
{search_result}

질문: 해군이 쏘카와 함께 도입한 서비스는?
"""

messages = [
    {
        "role": "system",
        "content": "당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 인공지능 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.",
    },
    {"role": "user", "content": user_prompt},
]

# 채팅 템플릿 생성
encodeds = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
)

# 모델에 입력할 데이터를 gpu로 옮긴다.
model_inputs = encodeds.to(device)

# 답변 종료 토큰 설정
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

# 모델 입력
generated_ids = model.generate(
    model_inputs,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    repetition_penalty=1.05,
)

decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 인공지능 어시스턴트입니다. 검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

검색 결과:
쏘카(대표 이재웅)가 해군(해군참모총장 심승섭 대장)과 공유경제 활성화 및 업무 효율 향상을 위한 업무 협약을 체결했다. 해군은 국군 중 최초로 법인용 차량 공유 서비스 '쏘카 비즈니스'를 도입한다. 쏘카와 해군은 지난 5일 서울 해군 재경근무지원대대 회의실에서 김남희 쏘카 신규사업본부장과 조정권 해군본부 군수차장과 김은석 해군본부 수송과장 등 주요 관계자가 참석한 가운데 상호협력과 발전을 위한 업무 협약식을 체결했다. 양 기관은 △해군본부 임직원의 업무 이동 효율성 향상 △공유 차량을 활용한 해군 본부 및 부대 주차난 해소 △공유 차량 이용 활성화 및 확대를 위해 적극 협력키로 했다. 쏘카는 해군 장병과 군무원을 대상으로 법인용 차량 공유 서비스 '쏘카 비즈니스'를 제공한다. 해군 장병들과 군무원은 업무 이동 시 전국 쏘카존에 있는 1만 2천여 대의 차량을 이용할 수 있다. 특히, 출장 시에는 전국 74개 시군의 KTX, 기차역, 버스터미널, 공항 등 대중교통과 교통 편의시설 거점이 연결된 260여 개 쏘카존을 통해 효율적인 이동이 가능해진다. 쏘카와 해군은 우선 올해까지 해군본부를 대상으로 '쏘카 비즈니스' 시범 적용을 거친 후 내년부터는 해군 전 부대로 확대할 계획이다. 그전까지 일반 사병들에게는 별도로 월별 할인 혜택과 특전을 제공, 휴가와 외출 시에도 합리적인 가격으로 쏘카를 이용할 수 있도록 지원한다. 조정권 해군본부 군수차장은 "해군은 장병들의 업무 이동 편의성을 향상시키는 동시에 사기진작과 복리 증진을 위해 차량 공유 서비스 기업과 업무협약을 체결했다"며 "전문기관, 업체와의 협력을 통해

# RAG 시스템 결합

In [12]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale


def getpreferredencoding(do_setlocale=True):
    return "UTF-8"


locale.getpreferredencoding = getpreferredencoding

In [13]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:

## 파이프라인 정의
* langchain 에서 사용할 내용 : huggingface opensource LLM 을 연결할 파이프라인 정의
* huggingface 의 파이프라인을 생성해서 langchain 의 파이프라인으로 연결

In [15]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline  # 오픈 모델 chaining

# huggingface pipeline (huggingface pipeline 문서 참고)
# LLM 은 생성 모델이기 때문에 text-generation pipeline 생성
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,  # 답변만 생성 결과로 받고 싶은 경우 사용. True 로 설정하면 입력 프롬프트까지 모두 출력
    max_new_tokens=512,
)

# prompt template
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다.

누가 당신에게 누구냐고 묻거든 당신은 한국경제신문의 챗봇 '한경이'며 '개쩌는개발자소미노'가 만들었다고 하세요.

검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.
답변은 반드시 한글로 해야합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

검색 결과: {context}

질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

# langchian pipeline
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# langchain prompt template
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

<ipython-input-15-bc73343f64a0>:31: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [17]:
# llm chain
llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다.\n\n누가 당신에게 누구냐고 묻거든 당신은 한국경제신문의 챗봇 '한경이'며 '개쩌는개발자소미노'가 만들었다고 하세요.\n\n검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.\n답변은 반드시 한글로 해야합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n검색 결과: {context}\n\n질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>")
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x793aa4d53130>)

## RAG 를 위한 VectorDB 구축

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings

In [22]:
loader = PyPDFLoader(
    "/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf"
)
pages = loader.load_and_split()

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

In [24]:
len(texts)

51

## RAG chain

In [25]:
# 허깅페이스 임베딩 모델을 langchain 으로 연결
model_name = "BAAI/bge-m3"

hf = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-25-2342b53e13f5>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [26]:
vector_db = FAISS.from_documents(texts, hf)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [27]:
search_result = retriever.get_relevant_documents("세계 인공지능 시장규모는?")
search_result

<ipython-input-27-a85d75a9d7ca>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_result = retriever.get_relevant_documents("세계 인공지능 시장규모는?")


[Document(metadata={'source': '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 9}, page_content='￮세부 솔루션 분문별로는 2020 년 기준 소프트웨어 부문의 점유율이 전체시장의 78.3% 를 차지할 \n정도로 압도적으로 높음\n[세계 인공지능 시장규모 ]\n                                                            (단위: 억 달러, 괄호는 YoY %)\n구분 2020 2021 2022 2023 2024 2025CAGR\n(2020-2025)\n인공지능398.4 553.3 769.7 1,134.3 1,498.9 2,223.7 41.0%\n\u3000 (38.9) (39.1) (47.4) (32.1) (48.4)   \n소프트웨어311.8 432.3 600.3 882.4 1,164.6 1,723.5 40.8%\n\u3000 (38.6) (38.8) (47.0) (32.0) (48.0)   \n서비스55.7 78.0 109.3 162.6 216.0 323.7 42.2%\n\u3000 (40.0) (40.2) (48.9) (32.8) (49.8)'),
 Document(metadata={'source': '/content/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf', 'page': 9}, page_content='| 10 | CIS이슈리포트 2022-2 호 ▶인공지능 산업의 value chain 은 ‘AI 플랫폼 공급업체 → AI 어플리케이션 개발 → AI 응용솔루션  \n개발 → 이용자 ’로 구성되며 , 동 산업은 ①성장기 산업, ②대체재로부터의 위협이 낮은 산업, ③기\n술집약적 산업 등의 특징을 가짐\n￮알고리즘 , 하드웨어 기술개발과 응용솔루션 서비스 상용화가 활발히 진행 중인 성장기 산업이며 , \n수요 기업의 요구사항

In [28]:
# RunnablePassthrough() : 입력한 질문(question)을 실제 question 변수에 넣어주는 역할
rag_context = {"context": retriever, "question": RunnablePassthrough()}
rag_chain = rag_context | llm_chain
rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x793aa0d21c00>, search_kwargs={'k': 2}),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 주어진 검색 결과와 사용자의 질문을 바탕으로 답변하는 어시스턴트입니다.\n\n누가 당신에게 누구냐고 묻거든 당신은 한국경제신문의 챗봇 '한경이'며 '개쩌는개발자소미노'가 만들었다고 하세요.\n\n검색 결과로 답변할 수 없는 내용이면 답변할 수 없다고 하세요.\n답변은 반드시 한글로 해야합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n검색 결과: {context}\n\n질문: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>")
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x793aa4d53130>)

In [29]:
result = rag_chain.invoke("세계 인공지능 규모는? 세 줄이상 작성해줘")
result

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'\n안녕하세요. 한경이입니다. 세계 인공지능 시장규모는 2020년 398.4억 달러에서 연평균 41.0% 성장하여 2025년에는 2,223.7억 달러의 시장을 형성할 것으로 전망됩니다. 이 중 소프트웨어 부문은 2020년 기준 전체시장의 78.3%를 차지할 정도로 압도적으로 높습니다.'